In [2]:
from utils.prepare_data import prepare_short_data, prepare_full_data
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = prepare_short_data()
data.head()

,Date,AFKS,AFLT,AGRO,AKRN,ALRS,AMEZ,APTK,AQUA,BANEP,...,TATNP_pct,TCSG_pct,TGKA_pct,TRNFP_pct,TTLK_pct,UPRO_pct,VKCO_pct,VSMO_pct,VTBR_pct,YNDX_pct
0,2018-01-03,12.350,140.4,570.0,3930.0,75.81,4.82,8.65,140.5,1357.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-04,12.385,148.9,565.0,3994.0,77.00,4.75,8.59,152.0,1364.5,...,0.002720,NaN,0.008207,0.020078,0.003460,-0.005064,NaN,0.014706,0.011111,0.017751
2,2018-01-05,12.390,147.3,562.5,3985.0,76.90,4.77,8.59,155.0,1368.5,...,0.008408,NaN,0.005698,-0.002187,0.020690,0.005482,NaN,0.002899,0.002997,-0.017947
3,2018-01-09,12.715,150.5,566.5,3982.0,76.79,4.80,8.50,162.0,1390.0,...,0.022593,NaN,0.010117,-0.016438,0.013514,0.012461,NaN,0.007514,0.011952,0.010553
4,2018-01-10,12.700,149.3,570.0,3995.0,76.70,4.88,8.40,160.0,1444.0,...,0.004734,NaN,0.004006,-0.004178,0.020000,-0.002308,NaN,0.001147,-0.005709,0.004585


In [4]:
data = prepare_full_data()

In [5]:
data.head()

,SECID,AFKS_close,AFLT_close,AGRO_close,AKRN_close,ALRS_close,AMEZ_close,APTK_close,AQUA_close,BANEP_close,...,VSMO_D3,VSMO_D4,VTBR_D1,VTBR_D2,VTBR_D3,VTBR_D4,YNDX_D1,YNDX_D2,YNDX_D3,YNDX_D4
1,2018-01-03 00:00:00,12.350,140.4,570.0,3930.0,75.81,4.82,8.65,140.5,1357.0,...,0.004135,0.000000,-0.054346,0.045848,0.045848,0.002330,-0.029828,0.012767,0.015894,0.002089
2,2018-01-04 00:00:00,12.385,148.9,565.0,3994.0,77.00,4.75,8.59,152.0,1364.5,...,0.018746,0.000000,-0.013944,0.011111,0.015556,0.007941,0.000257,0.018013,0.021616,0.000000
3,2018-01-05 00:00:00,12.390,147.3,562.5,3985.0,76.90,4.77,8.59,155.0,1368.5,...,0.013986,0.000583,-0.010236,-0.001591,0.000000,0.015552,0.001528,-0.011953,0.000000,0.023425
4,2018-01-09 00:00:00,12.715,150.5,566.5,3982.0,76.79,4.80,8.50,162.0,1390.0,...,0.005764,0.003470,-0.002376,0.008137,0.029371,0.001391,-0.007353,0.002810,0.012516,0.001791
5,2018-01-10 00:00:00,12.700,149.3,570.0,3995.0,76.70,4.88,8.40,160.0,1444.0,...,0.004028,0.001729,0.009901,-0.009608,0.005882,0.011905,0.000000,0.004585,0.008660,0.016309
